# Database Querying and Analysis with Langchain and OctoAI
---

## Getting Started

To work with this notebook locally, you can download the notebook using file/download.

## Overview
1. Clone and install the `octoml-llm-qa` repo to use OctoAI with Lanchain (next cell)
2. Load transactional data into a Dataframe & SQLite DB
3. Run a natural-language queries over your dataframe or DB with Lanchain calling an OctoAI-hosted LLM

### Setup
Install notebook dependencies including Langchain and the OctoAI extension of Lanchain.

In [ ]:
from IPython.display import clear_output, Javascript
import IPython

# Installs a wrapper for Lanchain to call OctoAI LLMs
! yes | rm -r octoml-llm-qa
! git clone -b db_demo https://github.com/octoml/octoml-llm-qa.git && cd octoml-llm-qa && pip install -e .

# Install other useful tools for this demo
! pip install gspread oauth2client pandas sqlalchemy

clear_output()


In [ ]:
# Clear output and restarts notebook
#IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

from IPython.display import clear_output, Javascript

clear_output()
print("Kernel will automatically restart and load newly installed packages.")


# Function to restart the kernel
def restart_kernel():
    display(Javascript('Jupyter.notebook.kernel.restart()'))

# Call the function to restart the kernel
restart_kernel()


Kernel will automatically restart and load newly installed packages.


<IPython.core.display.Javascript object>

### Kernel Restart  👀❗

The kernel must restart for each package to become usable.

This will only take a few seconds. Once finished, the cells after this block will run correctly.

---






# Populate a SQLite DB with dummy transactions data

In [ ]:
# Load transactional data from Google Sheets
SHEETS_URL = "https://docs.google.com/spreadsheets/d/1WwIQx7f-CNtX-h1wvmMlF_gMlPn4S3mCVmv_VLrLvJo/export?format=csv"
CSV_DATA = "transactions.csv"
! curl -L $SHEETS_URL -o transactions.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   385    0   385    0     0    332      0 --:--:--  0:00:01 --:--:--   332
100  4137    0  4137    0     0   2651      0 --:--:--  0:00:01 --:--:--  2651


In [ ]:
# Load data into a dataframe and a temporary database
import pandas as pd
import sqlite3
df = pd.read_csv(CSV_DATA)
df.head()


# Connect to a temporary SQLite database
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()


# Write the DataFrame to the SQLite database
df.to_sql('transactions', conn, if_exists='replace', index=False)

# Execute a query to get the list of databases
query = "SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';"
cursor.execute(query)

# Fetch the results
databases = cursor.fetchall()

# Print the list of databases
for database in databases:
    print(database[0])



transactions


In [ ]:
# Check our SQL DB connection

# Create a cursor object to execute SQL queries
conn = sqlite3.connect('my_database.db')
cursor = conn.cursor()

query = "PRAGMA table_info(transactions)"
cursor.execute(query)

# Fetch the results
columns = cursor.fetchall()

# Print the column names
for column in columns:
    print(column[1])

# Execute a query
query = "SELECT * FROM transactions"
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Process the results and print 5
for row in results[:5]:
    print(row)


# Close the cursor and connection
# Close the cursor and connection
cursor.close()
conn.close()

timestamp
product
customer
quantity
price
(1664644000, 'Product E', 'Customer 1', 8, 68.52)
(1666459000, 'Product A', 'Customer 2', 2, 76.51)
(1682529000, 'Product A', 'Customer 5', 7, 67.77)
(1678814000, 'Product E', 'Customer 1', 3, 16.87)
(1659115000, 'Product D', 'Customer 4', 9, 94.97)


### Query Sqlite with an OctoAI LLM & Lanchain's `SQLDatabaseChain`

<a href="https://octoai.cloud/settings/tokens/add">Generate a new OctoAI token here </a> and set the `OCTOAI_API_TOKEN=<your-token-here>` environment variable.

In [ ]:
# Import Lanchain and OctoAI tools
from octo_llms import OctoAiCloudLLM
from langchain import SQLDatabase, SQLDatabaseChain

# Set Token to Use OctoAI
%env OCTOAI_API_TOKEN=

# Create a new Langchain LLM instance for OctoML with an OctoAI endpoint URL (a public Dolly container)
endpoint_url = "https://dolly-demo-kk0powt97tmb.octoai.cloud/predict"
llm = OctoAiCloudLLM(endpoint_url=endpoint_url)

# Connect the Langchain agent to our DB
db_conn = "sqlite:///my_database.db"
db = SQLDatabase.from_uri(db_conn,include_tables=['transactions'])



ModuleNotFoundError: ignored

In [ ]:
from langchain.prompts.prompt import PromptTemplate

# Here, we can do a bit of prompt engineering to improve the quality of our results
_DEFAULT_TEMPLATE = """Given an input question, think step-by-step for how to first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Use the following format:


Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

{table_info}

Ignore the following columns:

timestamp

Question: {input}"""

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "dialect"], template=_DEFAULT_TEMPLATE
)

# Defines a wrapper function for the SQLDatabaseChain
def query_wrapper(query,max_tries=10):

    db_chain = SQLDatabaseChain.from_llm(llm,
                                     db,
                                     prompt=PROMPT,
                                     verbose=True)

    # Tries to construct a syntactically valid SQL query until you get it right
    while True and max_tries>=1:
        max_tries-=1
        try:
            db_chain(query)
            break
        except:
            continue

In [ ]:
# Returns the largest single transaction in the price column
max_price_query = """What is the largest value in the price column?"""

# Which customer has the most transactions?
most_transactions_per_customer="""Using the price column, what customer has s"""

# What is the average price of all transactions?
average_price_query = """What is the average price of all transactions?"""

# Which 2 customers have the most transactions?
most_transactions_per_customer_2="""Which 2 customers have the most transactions?"""

# Run the query until it returns a dict
query_wrapper(max_price_query)

### Pandas example - Note: this is much less predictable

In [ ]:
from langchain.agents import create_pandas_dataframe_agent

In [ ]:
# Defines a query function wrapper for a Dataframe
def df_llm(query,max_tries=10):
    # Flushes DB chain each time
    agent = create_pandas_dataframe_agent(OctoAiCloudLLM(endpoint_url=endpoint_url), df, verbose=True)


    while True and max_tries>=1:
        max_tries-=1
        try:
            agent.run(query)
            break
        except:
            continue

In [ ]:
df_llm(average_price_query)